In [ ]:
!git clone https://github.com/deepanshudashora/custom_models.git

import config
from custom_models.YOLOv3.utils import get_loaders

import warnings
warnings.filterwarnings("ignore")

from typing import Any, Union
from pytorch_lightning import LightningModule, Trainer, seed_everything
import torch 
from torchmetrics import Accuracy
import torch.optim as optim
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning import Trainer

import pandas as pd 
from custom_models.YOLOv3.loss import YoloLoss
loss_fn = YoloLoss()
from IPython.core.display import display
import seaborn as sn
from custom_models.YOLOv3.model import YOLOv3
from custom_models.YOLOv3.dataset import YOLODataModule
from custom_models.YOLOv3.main import YOLOTraining,check_accuracy
import os 


In [ ]:
print("/kaggle/input/pascal-voc-dataset-used-in-yolov3-video/PASCAL_VOC/" + "train.csv")
train_loader, test_loader, train_eval_loader = get_loaders(
    train_csv_path="/kaggle/input/pascal-voc-dataset-used-in-yolov3-video/PASCAL_VOC/"+ "train.csv", test_csv_path="/kaggle/input/pascal-voc-dataset-used-in-yolov3-video/PASCAL_VOC/" + "test.csv"
)

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

class AppendableTensorBoardLogger(TensorBoardLogger):
    def log_metrics(self, metrics, step):
        if self.version is None:
            self._get_experiment()
        assert self.version is not None, "experiment doesn't exist"
        
        # Log metrics using TensorBoard's add_scalars method
        for key, value in metrics.items():
            self.experiment.add_scalars(key, {self.version: value}, step)

    def close(self):
        pass  # Override the close method to prevent closing the experiment

In [ ]:
common_version = "version_0"  # Choose a version name

# Instantiate custom loggers with the same experiment version
tb_logger = AppendableTensorBoardLogger('/kaggle/working/results/tf_logs', version=common_version)
csv_logger = CSVLogger('/kaggle/working/results/csv_training_logs', version=common_version)  # Point to the directory containing existing CSV logs

max_epochs = 40
max_lr = 1.23E-02
learning_rate = config.LEARNING_RATE
weight_decay = config.WEIGHT_DECAY
check_val_every_n_epoch = 5

loss_fn = YoloLoss()
model = YOLOv3(num_classes=config.NUM_CLASSES).to(config.DEVICE)
model = YOLOTraining(loss_fn,config,model,max_lr,train_loader,check_accuracy,5/max_epochs)
data_module = YOLODataModule(train_loader, test_loader, train_eval_loader)
trainer = Trainer(
    max_epochs=max_epochs,
    accelerator="gpu",
    devices=1 if torch.cuda.is_available() else None,  # limiting got iPython runs
    logger=[tb_logger, csv_logger],
    precision=16,
    callbacks=[LearningRateMonitor(logging_interval="step"), TQDMProgressBar(refresh_rate=10)],
    check_val_every_n_epoch=check_val_every_n_epoch
)

ckpt_path = "/kaggle/working/results/tf_logs/lightning_logs/version_0/checkpoints/epoch=19-step=82760.ckpt"
trainer.fit(model, data_module, ckpt_path=ckpt_path)